# Upgrad Recommender Assignment EDA

## Setup

#### GDrive

In [1]:
import os
# give googe drive the required permission
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
os.chdir("/content/drive/MyDrive/pyspark/code_files/")
os.getcwd()

'/content/drive/MyDrive/pyspark/code_files'

#### Spark

In [3]:
try:
  import pyspark
except:
  !pip install pyspark==3.1.2
  import pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 18.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880756 sha256=f6e0ffac47bd949864d69cf05538055efad3d5e5f2f50b29ce1677ee879e80ff
  Stored in directory: /root/.cache/pip/wheels/ef/70/50/7882e1bcb5693225f7cc86698f10953201b48b3f36317c2d18
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [46]:
try:
  import lenskit
except:
  %pip install lenskit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.8/87.8 kB 8.6 MB/s eta 0:00:00


In [4]:
from pyspark.sql import SparkSession
from pyspark.context import SparkContext

In [5]:
spark = SparkSession.builder.master("local").config('spark.ui.port', '4050').getOrCreate()

In [6]:
spark

## Import Library

In [95]:
import pandas as pd
import numpy as np

# Import for typecasting columns
from pyspark.sql.types import IntegerType,BooleanType,DateType,FloatType,StringType
from pyspark.sql.types import ArrayType

# Pyspark Functions
from pyspark.sql.functions import regexp_replace, split, col, datediff, months_between, when, count, countDistinct, posexplode
from pyspark.sql.types import IntegerType, BooleanType, DateType, FloatType, StringType, ArrayType

# Model Functions
from lenskit import batch, topn, util
from lenskit.algorithms import Recommender, als, item_knn as knn
from pyspark.ml.recommendation import ALS

# Evaludation Functions
from pyspark.ml.evaluation import RegressionEvaluator

## Reading Data

In [8]:
raw_ratings_df = (spark.read.csv("raw_ratings_small.csv", # modify the path to read the data
                                 header=True,
                                 inferSchema= True))
raw_ratings_df.show(5)

+-------+---------+------+--------------------+-----------+
|user_id|recipe_id|rating|              review|review_date|
+-------+---------+------+--------------------+-----------+
| 483827|   306785|     5|Being a  huge fan...| 2008-07-15|
|   6258|    20930|     5|Jan  what an inte...| 2002-07-09|
| 102602|    20930|     5|Jan  we love your...| 2003-10-26|
| 296027|   182985|     4|Very nice fresh s...| 2007-05-19|
|   9580|   208980|     5|I saw this on 30 ...| 2007-02-05|
+-------+---------+------+--------------------+-----------+
only showing top 5 rows



In [9]:
raw_recipes_df = spark.read.csv("raw_recipies_small.csv", # modify the path to read the data
                                header=True,
                                inferSchema=True)
raw_recipes_df.show(5)

+--------------------+-----+-------+--------------+----------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+--------------+
|                name|   id|minutes|contributor_id| submitted|                tags|           nutrition|n_steps|               steps|         description|         ingredients|n_ingredients|year_of_review|
+--------------------+-----+-------+--------------+----------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+--------------+
|beat this  banana...|75452|     70|         15892|2003-11-04|['weeknight', 'ti...|[2669.3, 160.0, 9...|     12|['preheat oven to...|  from ann hodgman's|['sugar', 'unsalt...|            9|          2003|
|      chinese  candy|23933|     15|         35268|2002-03-29|['15-minutes-or-l...|[232.7, 21.0, 77....|      4|['melt butterscot...|a little differen...|['butterscotch ch...|     

In [10]:
raw_recipes_cleaned_df = spark.read.csv("RAW_recipes_cleaned.csv", # modify the path to read the data
                                header=True,
                                inferSchema=True)

In [11]:
raw_recipes_cleaned_df.show(5)

+--------------------+------+-------+--------------+----------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+
|                name|    id|minutes|contributor_id| submitted|                tags|           nutrition|n_steps|               steps|         description|         ingredients|n_ingredients|
+--------------------+------+-------+--------------+----------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+
|arriba   baked wi...|137739|     55|         47892|2005-09-16|['60-minutes-or-l...|[51.5, 0.0, 13.0,...|     11|['make a choice a...|autumn is my favo...|['winter squash',...|            7|
|a bit different  ...| 31490|     30|         26278|2002-06-17|['30-minutes-or-l...|[173.4, 18.0, 0.0...|      9|['preheat oven to...|this recipe calls...|['prepared pizza ...|            6|
|all in the kitche...|112140|    130|        

#### Code Check

In [12]:
assert (raw_recipes_df.count(), len(raw_recipes_df.columns)) == (20340, 13)
assert (raw_ratings_df.count(), len(raw_ratings_df.columns)) == (93357, 5)

## Data Cleaning

In [13]:
raw_recipes_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- minutes: integer (nullable = true)
 |-- contributor_id: integer (nullable = true)
 |-- submitted: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- nutrition: string (nullable = true)
 |-- n_steps: integer (nullable = true)
 |-- steps: string (nullable = true)
 |-- description: string (nullable = true)
 |-- ingredients: string (nullable = true)
 |-- n_ingredients: integer (nullable = true)
 |-- year_of_review: integer (nullable = true)



In [14]:
raw_ratings_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- recipe_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review: string (nullable = true)
 |-- review_date: string (nullable = true)



In [15]:
raw_recipes_df.select(*(count(when(col(c).isNull(), c)).alias(c) for c in raw_recipes_df.columns)).show()

+----+---+-------+--------------+---------+----+---------+-------+-----+-----------+-----------+-------------+--------------+
|name| id|minutes|contributor_id|submitted|tags|nutrition|n_steps|steps|description|ingredients|n_ingredients|year_of_review|
+----+---+-------+--------------+---------+----+---------+-------+-----+-----------+-----------+-------------+--------------+
|   0|  0|      0|             0|        0|   0|        0|      0|    0|        242|          0|            0|             0|
+----+---+-------+--------------+---------+----+---------+-------+-----+-----------+-----------+-------------+--------------+



In [16]:
raw_ratings_df.select(*(count(when(col(c).isNull(), c)).alias(c) for c in raw_ratings_df.columns)).show()

+-------+---------+------+------+-----------+
|user_id|recipe_id|rating|review|review_date|
+-------+---------+------+------+-----------+
|      0|        0|     0|     7|          0|
+-------+---------+------+------+-----------+



There are a few null values in the data, but we all the important values are present, so we are going to use all the data for analysis

Nutrition is a string column, which should be an array column <br>
Submitted is a string column, which should be a date column <br>
Tags is a string column, which should be an array column

## EDA

#### Nutrition Column into multiple columns

In [17]:
# Removing square brackets from string
raw_recipes_df = raw_recipes_df.withColumn('nutrition', regexp_replace('nutrition', "[\\[\\]]", ""))

In [18]:
# List of nutrition columns

nutrition_column_names = ['calories',
                          'total_fat_PDV',
                          'sugar_PDV',
                          'sodium_PDV',
                          'protein_PDV',
                          'saturated_fat_PDV',
                          'carbohydrates_PDV']

In [19]:
nutrition_cols_split = split(raw_recipes_df['nutrition'], ',')

# Write a loop to extract individual values from the nutrition column

for col_index, col_name in enumerate(nutrition_column_names):
    raw_recipes_df = raw_recipes_df.withColumn(col_name, nutrition_cols_split.getItem(col_index))
    raw_recipes_df = raw_recipes_df.withColumn(col_name, raw_recipes_df[col_name].cast(FloatType()))

In [20]:
# Standardize the nutrition values

for nutrition_col in nutrition_column_names:
    if nutrition_col!='calories':

        nutrition_per_100_cal_col = (nutrition_col.replace('_PDV','') + '_per_100_cal')
        raw_recipes_df = raw_recipes_df.withColumn(nutrition_per_100_cal_col,
                                               col(nutrition_col)*100/col("calories"))

        raw_recipes_df = raw_recipes_df.na.fill(value=0,subset=[nutrition_per_100_cal_col])

### Convert the tags column from a string to an array of strings

In [21]:
raw_recipes_df = raw_recipes_df.withColumn('tags', regexp_replace('tags', "[\\[\\]\\']", "")).withColumn('tags', regexp_replace('tags', " ", "")).withColumn('tags', split('tags', ','))

### Join both the dataframes using the recipie IDs.

In [22]:
interaction_level_df = raw_ratings_df.join(raw_recipes_df,
                                           raw_ratings_df.recipe_id ==  raw_recipes_df.id,
                                           "left"
                                           )

### Creating time based features

In [23]:
# Converting columns to datetype
interaction_level_df = (interaction_level_df
                        .withColumn('submitted', interaction_level_df['submitted'].cast(DateType())
                                   )
                        .withColumn('review_date', interaction_level_df['review_date'].cast(DateType())
                                   )
                       )

In [24]:
interaction_level_df = (interaction_level_df
                        .withColumn('days_since_submission_on_review_date',
                                     datediff(col('review_date'),col('submitted'))
                                   )
                        .withColumn('months_since_submission_on_review_date',
                                     months_between(col('review_date'),col('submitted'))
                                   )
                        .withColumn('years_since_submission_on_review_date',
                                     months_between(col('review_date'),col('submitted'))/12
                                   )
                         )

In [25]:
interaction_level_df.show(5)

+-------+---------+------+--------------------+-----------+--------------------+-----+-------+--------------+----------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+--------------+--------+-------------+---------+----------+-----------+-----------------+-----------------+---------------------+------------------+------------------+-------------------+-------------------------+-------------------------+------------------------------------+--------------------------------------+-------------------------------------+
|user_id|recipe_id|rating|              review|review_date|                name|   id|minutes|contributor_id| submitted|                tags|           nutrition|n_steps|               steps|         description|         ingredients|n_ingredients|year_of_review|calories|total_fat_PDV|sugar_PDV|sodium_PDV|protein_PDV|saturated_fat_PDV|carbohydrates_PDV|total_fat_per_100_cal| sugar_per_100_cal|sodi

## Feature Selection

Removing features which are text, and cannot be used to as of now: <br>
['review', 'name', 'id', 'contributer_id', 'submitted', 'tags', 'nutrition', 'Steps', 'description', 'year of review', 'total_fat_PDV', 'sugar_PDV',
'sodium_PDV', 'protein_PDV', 'saturated_fat_PDV', 'carbohydrates_PDV', 'ingredients']

In [26]:
interaction_level_df = interaction_level_df.drop('review', 'name', 'id', 'contributer_id', 'submitted', 'tags', 'nutrition', 'Steps', 'description', 'year of review', 'total_fat_PDV', 'sugar_PDV',
'sodium_PDV', 'protein_PDV', 'saturated_fat_PDV', 'carbohydrates_PDV', 'ingredients')

In [27]:
# Checking for null values
interaction_level_df.select(*(count(when(col(c).isNull(), c)).alias(c) for c in interaction_level_df.columns)).show()

+-------+---------+------+-----------+-------+--------------+-------+-------------+--------------+--------+---------------------+-----------------+------------------+-------------------+-------------------------+-------------------------+------------------------------------+--------------------------------------+-------------------------------------+
|user_id|recipe_id|rating|review_date|minutes|contributor_id|n_steps|n_ingredients|year_of_review|calories|total_fat_per_100_cal|sugar_per_100_cal|sodium_per_100_cal|protein_per_100_cal|saturated_fat_per_100_cal|carbohydrates_per_100_cal|days_since_submission_on_review_date|months_since_submission_on_review_date|years_since_submission_on_review_date|
+-------+---------+------+-----------+-------+--------------+-------+-------------+--------------+--------+---------------------+-----------------+------------------+-------------------+-------------------------+-------------------------+------------------------------------+-------------------

## Task 1: Train Test Split

In [28]:
# No of rows requried for training
num_review_int = interaction_level_df.count()
test_num_reviews_int = round(num_review_int *0.2)
test_num_reviews_int


18671

#### Finding date where to split

In [29]:
temp_ratings_df = (interaction_level_df.sort("review_date", ascending=False)
                                 .limit(test_num_reviews_int)
                  )

In [30]:
assert temp_ratings_df.count()  == 18671

In [31]:
cutoff_date = temp_ratings_df.collect()[-1][3]
cutoff_date

datetime.date(2011, 7, 17)

#### Testing and Training data

In [72]:
# All interactions which were rated BEFORE '2011-07-17' will be train data.

train_ratings_df  = interaction_level_df.filter(
                                             interaction_level_df.review_date < cutoff_date
                                           )

In [73]:
# All interactions which were rated ON OR AFTER '2011-07-17' will be test data.

test_ratings_df  = interaction_level_df.filter(
                                            interaction_level_df.review_date >= cutoff_date
                                           )

In [34]:
# no of columns are different as, I have already filtered for some columns for training and testing

assert (test_interaction_level_all_recipies_df.count(), len(test_interaction_level_all_recipies_df.columns)) == (18684, 19)
assert (train_interaction_level_df.count(), len(train_interaction_level_df.columns)) == (74673, 19)

## Task 2: Collaborative Filtering Model

In [44]:
# Count the total number of ratings in the dataset
numerator = train_ratings_df.select("rating").count()

# Count the number of distinct userIds and distinct recipe_Ids
num_users = train_ratings_df.select("user_id").distinct().count()
num_recipes = train_ratings_df.select("recipe_id").distinct().count()

# Set the denominator equal to the number of users multiplied by the number of movies
denominator = num_users * num_recipes

# Divide the numerator by the denominator
sparsity = (1.0 - (numerator *1.0)/denominator)*100
print("The training dataframe is ", "%.7f" % sparsity + "% empty.")

The training dataframe is  99.9829264% empty.


### Model Buiding

In [78]:
def manual_recommendation_check (user_id):
  '''
  Given a user ID form the test dataset, this function will return the names of the recipes recommended to the user.

  Initialize this function after the all_recs data frame is calculated.

  Input user_id of a user from the test set as an integer.
  Prints the names of recipes recommended to this user.
  Returns nothing.
  '''
  recs_user = test_predictions_unseen[test_predictions_unseen.user == user_id]
  recs_user_list = list(recs_user.item.values)
  recs_user_list = [x.item() for x in recs_user_list]
  display((raw_recipes_df.filter(F.col('id').isin(recs_user_list))
               .select("name")
               .collect(10)
               ))

In [ ]:
def manual_recommendation_check (user_id):

  recs_user = all_recs_als[all_recs_als.user == user_id]
  recs_user_list = list(recs_user.item.values)
  recs_user_list = [x.item() for x in recs_user_list]
  display((raw_recipes_df.filter(F.col('id').isin(recs_user_list))
               .select("name")
               .collect(10)
               ))

In [69]:
als_model = ALS(userCol= "user_id", itemCol= "recipe_id", ratingCol= "rating", nonnegative = True, implicitPrefs = False, coldStartStrategy="drop")

In [70]:
assert type(als_model) == pyspark.ml.recommendation.ALS

#### Model Fitting and Training

In [74]:
model = als_model.fit(train_ratings_df)

### Model Predictions

In [75]:
# use the model to create predictions for test data
test_predictions_unseen = model.transform(test_ratings_df)

In [76]:
assert test_predictions_unseen.select(col("prediction"))

In [82]:
test_predictions_unseen.select(col("user_id"),
                               col("recipe_id"),
                               col("rating"),
                               col("prediction")
                              ).show(5)

+-------+---------+------+----------+
|user_id|recipe_id|rating|prediction|
+-------+---------+------+----------+
| 199020|    55265|     5|  4.411028|
| 369284|    76143|     5| 3.6750858|
| 224235|    89385|     5|   4.41608|
| 385423|    95476|     4| 3.4344254|
| 538098|    95476|     5| 4.3493814|
+-------+---------+------+----------+
only showing top 5 rows



#### Recommenation from model

In [83]:
# Making 10 recommendation for each user

recommendations = model.recommendForAllUsers(10)

In [85]:
assert len(dict(recommendations.select(col("recommendations")).collect()[0][0])) == 10

In [88]:
# Transfroming the results by exploding the recommendations column

recommendations = (recommendations.select(col("user_id"),
                                          posexplode(F.col("recommendations")).alias("pos", "item"))
                                  .select(col("user_id"),
                                          col("pos"),
                                          col("item.recipe_id").alias("recomended_recipe_id"),
                                          col("item.rating").alias("predicted_rating")))

In [89]:
recommendations.show(5)

+-------+---+--------------------+----------------+
|user_id|pos|recomended_recipe_id|predicted_rating|
+-------+---+--------------------+----------------+
|  28170|  0|              146694|       6.1493006|
|  28170|  1|               65833|        5.973863|
|  28170|  2|              156550|       5.9331894|
|  28170|  3|              108417|       5.9030004|
|  28170|  4|              360099|       5.8052964|
+-------+---+--------------------+----------------+
only showing top 5 rows



## Task 4: Model Evaluation

### RMSE

In [92]:
# Define evaluator as RMSE and print RMSE value
evaluator_seen = RegressionEvaluator(metricName="rmse",
                                     labelCol= "rating",
                                     predictionCol= "prediction"
                                     )

In [93]:
RMSE = evaluator_seen.evaluate(test_predictions_unseen) # Use the evaluator to find the RMSE on the test set.
print(RMSE)

1.4124921079602855


### Rank based Metrics

In [94]:
# Converting recommendation to pandas df
all_recs_als = recommendations.toPandas()

In [97]:
column_names = {'user_id': 'user', 'pos': 'rank', 'recomended_recipe_id': 'item', 'predicted_rating':	'score'}
all_recs_als = all_recs_als.rename(columns=column_names)

In [98]:
all_recs_als["algorithm"] = "ALS"

In [99]:
all_recs_als.head()

,user,rank,item,score,algorithm
0,28170,0,146694,6.149301,ALS
1,28170,1,65833,5.973863,ALS
2,28170,2,156550,5.933189,ALS
3,28170,3,108417,5.903000,ALS
4,28170,4,360099,5.805296,ALS


In [101]:
# Convert the test dataset to pandas and ensure that it has the same column names as shown in the cell below.
# Also, note there are only three columns in the rest data. You have to ensure your test data looks identical.

test_data = test_ratings_df.select("user_id", "recipe_id", "rating").toPandas()
test_data.rename(columns = {"user_id":"user", "recipe_id":"item", "rating":"rating"}, inplace = True)
test_data.head()

,user,item,rating
0,1803719199,278119,5
1,2002330544,278119,4
2,1239906,234183,5
3,286566,229794,5
4,2001454217,232046,4


In [103]:
rla = topn.RecListAnalysis()
rla.add_metric(topn.recip_rank)
rla.add_metric(topn.ndcg)
rla.add_metric(topn.dcg)
results = rla.compute(all_recs_als, test_data)
results.head()

nrecs  recip_rank  ndcg  dcg
algorithm user                                
ALS       28170      10         0.0   NaN  NaN
          56680      10         0.0   0.0  0.0
          108460     10         0.0   0.0  0.0
          139830     10         0.0   NaN  NaN
          198430     10         0.0   NaN  NaN

In [105]:
results = results.reset_index()

In [115]:
results[results.ndcg > 0]

,algorithm,user,nrecs,recip_rank,ndcg,dcg
20473,ALS,593927,10,0.5,0.085188,3.154649


In [116]:
results[results.dcg > 0]

,algorithm,user,nrecs,recip_rank,ndcg,dcg
20473,ALS,593927,10,0.5,0.085188,3.154649


In [117]:
results[results.recip_rank > 0]

,algorithm,user,nrecs,recip_rank,ndcg,dcg
20473,ALS,593927,10,0.5,0.085188,3.154649


Only one record with ndcg and dcg value greater than 0